# Tensorflow

- 일반적인 프로그래밍과는 약간 다른 개념을 가지고 있음
    - 텐서(tensor)
    - 플레이스홀더(placeholder)
    - 변수(variable)
    - 연산
- 언어속의 또 다른 언어같은 느낌..

# 텐서와 그래프 실행

In [1]:
import tensorflow as tf

In [2]:
hello = tf.constant('Hello, TensorFlow!')
print(hello)

Tensor("Const:0", shape=(), dtype=string)


## constant
tf.constant로 상수를 선언후 출력하면  
```Tensor("Const:0", shape=(), dtype=string)```  
처럼 나오는데 **Tensor**형이고 string타입의 데이터라는 것을 알려줌  
constant는 일반적인 변수 선언과 같음  
tensorflow는 또 다른 언어같은 느낌이니 여기서 hello라는 python변수에 tensorflow변수를 박아뒀다라고 봐야될지..?  

- 텐서는 텐서플로에서 다양한 수학식을 계산하기 위한 가장 기본적인 자료형
- **rank**와 **shape**라는 개념을 가지고 있음

    - 3 - rank가 0인 텐서. shape는 []
    - [1,2,3] - rank가 1인 텐서. shape는 [3]
    - [[1,2,3],[4,5,6]] - rank가 2인 텐서. shape는 [2,3]
    - [[[1,2,3]],[[7,8,9]]] - rank가 3인 텐서. shape는 [2,1,3]

- rank는 차원의 수. shape는 요소의 갯수를 뜻함
- rank가 0이면 스칼라, 1이면 벡터, 2면 행렬, 3이면 n-Tensor 또는 n차원 텐서라함

In [3]:
a = tf.constant(10)
b = tf.constant(32)
c = tf.add(a, b)  # a + b 로도 쓸 수 있음
print(c)

Tensor("Add:0", shape=(), dtype=int32)


## 그래프
출력시 42가 아닌
```Tensor("Add:0", shape=(), dtype=int32)``` 이 출력되는 이유는  
텐서플로 프로그램의 구조가 두가지로 분리되어 있기 때문
1. 그래프 생성
```
    a = tf.constant(10)
    b = tf.constant(32)
    c = tf.add(a, b)
```
2. 그래프 실행(아래 코드)
```
    print(sess.run(hello))
    print(sess.run([a, b, c]))
```

그래프는 텐서들의 연산 모음이며  
필요할 때 연산을 실행하는 코드를 넣어 '**원하는 시점**'에  실제 연산을 수행함  
이를 **지연 실행(lazy evaluation)**이라고 하며 함수형 프로그래밍에서 많이 사용  
실제 계산은 C++로 구현한 코어라이브러리에서 수행. 파이썬에서 동작하지만 빠른이유

In [4]:
sess = tf.Session()
print(sess.run(hello))
print(sess.run([a, b, c]))
sess.close()

b'Hello, TensorFlow!'
[10, 32, 42]


In [5]:
with tf.Session() as sess:
    print(sess.run(hello))
    print(sess.run([a, b, c]))

b'Hello, TensorFlow!'
[10, 32, 42]


그래프의 실행은 Session 안에서 이뤄져야하고 Session 객체와 run을 이용하면 됨  
세션을 열고 닫는게 싫다면 위와 같은 코드로  

---------

# 플레이스홀더와 변수

아직까지도 혼란스러운 개념임..  
- 플레이스홀더 - 그래프에 사용할 입력값을 나중에 받기 위해 사용하는 매개변수(parameter)와 비슷함
- 변수 - ㄹㅇ 변하는수. 프로그래밍에서 매개변수의 그 변수가 아닌 적절한 값을 찾아 학습시마다 계속 변하는 수

In [6]:
X = tf.placeholder(tf.float32, [None, 3])
print(X)

Tensor("Placeholder:0", shape=(?, 3), dtype=float32)


```Tensor("Placeholder:0", shape=(?, 3), dtype=float32)```  
Placeholder형이며 (?,3)모양의 float32형태의 데이터가 생성됨  
여기서 (?,3)이란 [None, 3]으로 설정해서 열은 3으로 정해졌지만 받아들일 행의 크기가 정해지지 않았다는 뜻  

In [7]:
x_data = [[1, 2, 3], [4, 5, 6]]

X 플레이스홀더에 넣을 값

In [8]:
W = tf.Variable(tf.random_normal([3, 2]))
b = tf.Variable(tf.random_normal([2, 1]))

Instructions for updating:
Colocations handled automatically by placer.


tf.Variable: 그래프를 계산하면서 최적화 할 변수  
tf.random_normal: 각 변수들의 초기값을 정규분포 랜덤 값으로 초기화

In [9]:
expr = tf.matmul(X, W) + b

입력값과 변수들을 계산할 수식을 작성  
tf.matmul 처럼 mat* 로 되어 있는 함수로 행렬 계산을 수행

### 행렬곱
- 행렬곱 R1C1 x R2C2(R = Row 행, C = Column 열)에 대해 C1의 갯수는 = R2의 갯수 여야함
- 결과는 R1의 갯수 x C2의 갯수로 나옴

In [10]:
sess = tf.Session()
# Variable 들의 값들을 초기화 하기 위해 tf.global_variables_initializer 를 한 번 실행해야 함.
sess.run(tf.global_variables_initializer())

print("=== x_data ===")
print(x_data)
print("=== W ===")
print(sess.run(W))
print("=== b ===")
print(sess.run(b))
print("=== expr ===")
# feed_dict는 그래프를 실행할 때 사용할 입력값을 지정하는 것임
# 여기서는 X x W + b의 X에 x_data박는다는뜻
print(sess.run(expr, feed_dict={X: x_data}))

sess.close()

=== x_data ===
[[1, 2, 3], [4, 5, 6]]
=== W ===
[[ 1.8680998   0.29095984]
 [-0.23848693 -0.20896561]
 [ 0.95428354  1.1176282 ]]
=== b ===
[[ 2.161185  ]
 [-0.18224257]]
=== expr ===
[[ 6.415162   5.3870983]
 [11.823423   6.6425385]]


-----------------------------

# 선형회귀

- 주어진 x와 y값을 가지고 서로 간의 관계를 파악하는 것
- 입력에 대해 출력을 예측하는것. 머신러닝의 기본

In [11]:
import tensorflow as tf

x_data = [1,2,3]
y_data = [1,2,3]

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

# random_uniform - 균등분포. 여기서는 -1 ~ 1사이의 값을 갖는 무작위값으로 초기화
W = tf.Variable(tf.random_uniform([1], -1.0, 1.0))
b = tf.Variable(tf.random_uniform([1], -1.0, 1.0))

# W는 가중치 weight
# b는 편향 bias라고도 함
#hypothesis = tf.matmul(X, W) + b
hypothesis = X * W + b

## 손실함수(loss function)
한 쌍(x, y)의 데이터에 대한 **손실값**을 계산하는 함수  
실제값과 예측값이 얼마나 차이가 있는지 나타내는 값을 뜻함  

손실을 전체 데이터에 대해 구한 경우 이를 **비용**이라고 함  
**학습**이란 변수값을 다양하게 넣어 손실값을 최소화 하는 W와 b의 값을 구하는 것  

In [12]:
# 손실 함수를 작성
# mean(h - Y)^2 : 예측값과 실제값의 거리를 비용(손실) 함수로 정함
cost = tf.reduce_mean(tf.square(hypothesis - Y))
# 텐서플로우에 기본적으로 포함되어 있는 함수를 이용해 경사 하강법 최적화를 수행
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.05)
# 비용을 최소화 하는 것이 최종 목표
train_op = optimizer.minimize(cost)

Instructions for updating:
Use tf.cast instead.


In [13]:
### 수정중
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    # 최적화를 100번 수행
    for step in range(300):
        # sess.run 을 통해 train_op 와 cost 그래프를 계산
        # 이 때, 가설 수식에 넣어야 할 실제값을 feed_dict 을 통해 전달
        _, cost_val = sess.run([train_op, cost], feed_dict={X: x_data, Y: y_data})
        
        if step % 50 == 0: print(step, cost_val, sess.run(W), sess.run(b))

    print("\n=== Test ===")
    print("X: 5, Y:", sess.run(hypothesis, feed_dict={X: 5}))
    print("X: 2.5, Y:", sess.run(hypothesis, feed_dict={X: 2.5}))

0 10.1233635 [0.23794448] [0.24063179]
50 0.010210205 [0.88405246] [0.26357594]
100 0.003046954 [0.93666023] [0.14398627]
150 0.0009092707 [0.9653988] [0.07865676]
200 0.00027134715 [0.98109806] [0.0429686]
250 8.0976686e-05 [0.98967427] [0.02347287]

=== Test ===
X: 5, Y: [4.9844317]
X: 2.5, Y: [2.4987051]
